In [1]:
import pandas as pd
import numpy as np

class DataFrameProcessor:
    def __init__(self, changing_cats_path, categories_for_extraction_path, output_path):
        self.changing_cats_path = changing_cats_path
        self.categories_for_extraction_path = categories_for_extraction_path
        self.output_path = output_path
        self.changing_cats = None
        self.categories_for_extraction = None

    def load_data(self):
        """Загрузка данных из CSV файлов."""
        try:
            self.changing_cats = pd.read_csv(self.changing_cats_path)
            self.categories_for_extraction = pd.read_csv(self.categories_for_extraction_path)
            print("Данные загружены.")
        except Exception as e:
            print(f"Ошибка при загрузке данных: {e}")

    def clean_data(self):
        """Очистка данных."""
        try:
            # Удаление ненужных столбцов, если они существуют
            cols_to_drop = ["Unnamed: 3", "Unnamed: 4", "Unnamed: 5"]
            self.changing_cats.drop(columns=[col for col in cols_to_drop if col in self.changing_cats.columns], inplace=True)

            # Удаление строк с ненужными категориями
            unwanted_categories = ["Юмор", "Разное"]
            self.changing_cats = self.changing_cats[~self.changing_cats['Ours'].isin(unwanted_categories)]
            self.categories_for_extraction = self.categories_for_extraction[~self.categories_for_extraction['category'].isin(unwanted_categories)]

            print("Очистка данных завершена.")
        except KeyError as e:
            print(f"Ошибка: Не удается найти столбец: {e}")

    def update_tags(self):
        """Обновление тегов на основе категории."""
        try:
            # Создание словарей для замены категорий и тегов
            category_to_dict = self.changing_cats.set_index('Ours')['Rutube lvl0'].to_dict()
            tag_to_dict = self.changing_cats.set_index('Ours')['Tag'].to_dict()

            # Обработка категории "Охота и рыбалка"
            mask = self.categories_for_extraction['category'] == "Охота и рыбалка"
            
            # Только строки с категорией "Охота и рыбалка" изменяем на "Спорт" и добавляем тег "Рыбалка"
            self.categories_for_extraction.loc[mask, 'category'] = "Спорт"
            self.categories_for_extraction.loc[mask, 'tag'] = "Рыбалка"

            # Использование map() для замены категорий и тегов на основе словарей
            self.categories_for_extraction['category'] = self.categories_for_extraction['category'].map(
                category_to_dict).fillna(self.categories_for_extraction['category'])
            self.categories_for_extraction['tag'] = self.categories_for_extraction['category'].map(
                tag_to_dict).fillna(self.categories_for_extraction['tag'])

            print("Обновление тегов завершено.")
        except Exception as e:
            print(f"Ошибка при обновлении тегов: {e}")

    def save_to_csv(self):
        """Сохранение итогового DataFrame в CSV файл."""
        try:
            self.categories_for_extraction.to_csv(self.output_path, index=False, encoding='utf-8')
            print(f"Результаты сохранены в {self.output_path}.")
        except Exception as e:
            print(f"Ошибка при сохранении файла: {е}")

    def process(self):
        """Основной метод для выполнения всех этапов."""
        self.load_data()
        self.clean_data()
        self.update_tags()
        self.save_to_csv()
        return self.categories_for_extraction



In [4]:
# Параметры для инициализации класса
changing_cats_path = "/root/backend/data_for_renaming/rutube_refactor.csv"
categories_for_extraction_path = "~/mnt/nfs/data_our/keydata.csv"
output_path = "~/mnt/nfs/data_our/keydata_renamed.csv"           #Это готовый результат того что мы качали

# Создание и запуск экземпляра класса
processor = DataFrameProcessor(changing_cats_path, categories_for_extraction_path, output_path)
processor.process()

Ошибка при загрузке данных: [Errno 2] No such file or directory: '/root/mnt/nfs/data_our/keydata.csv'


TypeError: 'NoneType' object is not subscriptable

In [5]:
import os
import pandas as pd

# Путь к файлу, который нужно удалить
wrong_meta = "/mnt/nfs/data_our/metadata1.csv"

# Удаление файла
if os.path.exists(wrong_meta):
    os.remove(wrong_meta)

# Чтение данных
wright_meta = pd.read_csv("~/mnt/nfs/data_our/metadata.csv")
key_data = pd.read_csv("~/mnt/nfs/data_our/keydata_renamed.csv")

# Объединение датафреймов по категориям (снизу)
merged_csvs = pd.concat([wright_meta, key_data], ignore_index=True)

# Вывод информации о новом датафрейме
merged_csvs.info()


FileNotFoundError: [Errno 2] No such file or directory: '/root/mnt/nfs/data_our/metadata.csv'